In [ ]:
import string
import random
import nltk
import copy
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('reuters')
from nltk.corpus import reuters
from nltk import FreqDist
nltk.download('gutenberg')
from nltk.corpus import gutenberg
from nltk import ngrams
from nltk.corpus import stopwords
from collections import defaultdict

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [ ]:
emma_sents = gutenberg.sents('austen-emma.txt')
emma_words = gutenberg.words('austen-emma.txt')

In [ ]:
stop_words = set(stopwords.words('english'))
string.punctuation = string.punctuation +'"'+'"'+'-'+'''+'''+'â€”'
# string.punctuation
removal_list = list(stop_words) + list(string.punctuation)+ ['lt','rt']
# removal_list

In [ ]:
unigram=[]
bigram=[]

In [ ]:
tokenized_text=[]
for sentence in emma_sents:
  sentence = list(map(lambda x:x,sentence))
  for word in sentence:
    unigram.append(word)
  tokenized_text.append(sentence)
  bigram.extend(list(ngrams(sentence, 2,pad_left=True, pad_right=True)))
bigram

[(None, '['),
 ('[', 'Emma'),
 ('Emma', 'by'),
 ('by', 'Jane'),
 ('Jane', 'Austen'),
 ('Austen', '1816'),
 ('1816', ']'),
 (']', None),
 (None, 'VOLUME'),
 ('VOLUME', 'I'),
 ('I', None),
 (None, 'CHAPTER'),
 ('CHAPTER', 'I'),
 ('I', None),
 (None, 'Emma'),
 ('Emma', 'Woodhouse'),
 ('Woodhouse', ','),
 (',', 'handsome'),
 ('handsome', ','),
 (',', 'clever'),
 ('clever', ','),
 (',', 'and'),
 ('and', 'rich'),
 ('rich', ','),
 (',', 'with'),
 ('with', 'a'),
 ('a', 'comfortable'),
 ('comfortable', 'home'),
 ('home', 'and'),
 ('and', 'happy'),
 ('happy', 'disposition'),
 ('disposition', ','),
 (',', 'seemed'),
 ('seemed', 'to'),
 ('to', 'unite'),
 ('unite', 'some'),
 ('some', 'of'),
 ('of', 'the'),
 ('the', 'best'),
 ('best', 'blessings'),
 ('blessings', 'of'),
 ('of', 'existence'),
 ('existence', ';'),
 (';', 'and'),
 ('and', 'had'),
 ('had', 'lived'),
 ('lived', 'nearly'),
 ('nearly', 'twenty'),
 ('twenty', '-'),
 ('-', 'one'),
 ('one', 'years'),
 ('years', 'in'),
 ('in', 'the'),
 ('the',

In [ ]:
freq_bi = FreqDist(bigram)
freq_bi

FreqDist({('.', None): 5110, (',', 'and'): 1879, (None, '"'): 1555, ('."', None): 1156, ('Mr', '.'): 1153, ("'", 's'): 932, (';', 'and'): 866, ('Mrs', '.'): 699, (None, 'I'): 650, ('to', 'be'): 595, ...})

In [ ]:
d = defaultdict(int)
for a, b in freq_bi:
  if(a != None and b!= None):
    d[a] += freq_bi[a, b]

In [ ]:
p = defaultdict(int)
for a,b in freq_bi:
  if(a!= None and b!=None):
    p[a,b] = freq_bi[a,b] / d[a]

In [ ]:
def edits0(word): 
    "Return all strings that are zero edits away from word (i.e., just word itself)."
    return {word}


def splits(word):
    "Return a list of all possible (first, rest) pairs that comprise word."
    return [(word[:i], word[i:]) 
            for i in range(len(word)+1)]

def edits1(word):
    "Return all strings that are one edit away from this word."
    pairs      = splits(word)
    deletes    = [a+b[1:]           for (a, b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a, b) in pairs if len(b) > 1]
    replaces   = [a+c+b[1:]         for (a, b) in pairs for c in alphabet if b]
    inserts    = [a+c+b             for (a, b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    "Return all strings that are two edits away from this word."
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

alphabet = 'abcdefghijklmnopqrstuvwxyz'

In [ ]:
def random_test(sents,words,p):

  unique_words = set(words)
  test = sents[random.randint(0,len(sents)-1)]
  # print(*test)
  rand_ind = random.randrange(0,len(test))
  temp = (edits0(test[rand_ind]).union(edits1(test[rand_ind])).union(edits2(test[rand_ind])))
  real_nonreal = random.randint(1,5)
  if real_nonreal == 1:
    temp = temp.intersection(unique_words)
  else:
    temp = temp.difference(unique_words)
  temp = list(temp)
  chosen_index = random.randrange(0,len(temp))

  test_original = test.copy()

  test.pop(rand_ind)
  test.insert(rand_ind,temp[chosen_index])

  test_wrong = test.copy()

  

  ind1 = 0
  temp1 = 1
  for i in range(len(test)-1):
    if p[test[i],test[i+1]] < temp1:
      ind1 = i
      temp1 = p[test[i],test[i+1]]
  
  ind2 = 0
  temp2 = 1
  for i in range(1,len(test)):
    if p[test[len(test)-i-1],test[len(test)-i]] < temp2:
      ind2 = len(test)-i
      temp2 = p[test[len(test)-i-1],test[len(test)-i]]

  if ind2-ind1 ==2:
    ind = [ind2-1]
  else:
    ind = []
    for i in range(min(ind1,ind2),max(ind1,ind2)+1):
      ind.append(i)
  
  # print("ind:" ,ind)

  alphabet = 'abcdefghijklmnopqrstuvwxyz'

  candidates = dict()

  for i in ind:
    candidates[i] = (edits0(test[i]).union(edits1(test[i])).union(edits2(test[i])))

  
  for i in range(len(candidates)):
    temp = candidates[ind[i]].intersection(unique_words)
    temp = list(temp)
    candidates[ind[i]] = temp

  ind_max = 0
  temp = 0
  candidate = ''


  for index in ind:
    if index != 0 and index != len(test)-1:
      for i in range(len(candidates[index])):
        if p[test[index-1],candidates[index][i]]*p[candidates[index][i],test[index+1]] > temp:
          candidate = candidates[index][i]
          temp = p[test[index-1],candidates[index][i]]*p[candidates[index][i],test[index+1]]
          ind_max = index
    elif index == 0:
      for i in range(len(candidates[index])):
        if p[candidates[index][i],test[index+1]] > temp:
          candidate = candidates[index][i]
          temp = p[candidates[index][i],test[index+1]]
          ind_max = index
    else:
      for i in range(len(candidates[index])):
        if p[test[index-1],candidates[index][i]] > temp:
          candidate = candidates[index][i]
          temp = p[test[index-1],candidates[index][i]]
          ind_max = index
  # if real_nonreal == 1:
  #   print("real word error")
  # else: 
  #   print("nonreal word error")
  # print(ind_max)
  # print(candidate)
  # print("original: ", *test)
  test.pop(ind_max)
  test.insert(ind_max,candidate)
  # print("corrected: ", *test)
  return test_original, test_wrong, test







In [ ]:
count = 0
for i in range(20):
  x,y,z =random_test(emma_sents,emma_words,p)
  print(*x)
  print(*y)
  print(*z)
  if x == z:
    count+=1

Highbury , the large and populous village , almost amounting to a town , to which Hartfield , in spite of its separate lawn , and shrubberies , and name , did really belong , afforded her no equals .
Highbury , the large and populous village , almost amounting to a town , to which Hartfield , kf spite of its separate lawn , and shrubberies , and name , did really belong , afforded her no equals .
Highbury , the large and populous village , almost amounting to a town , to which Hartfield , in spite of its separate lawn , and shrubberies , and name , did really belong , afforded her no equals .
Has Emma read it all ?
Has Emma read ate all ?
Has Emma read it all ?
" Jane caught a bad cold , poor thing !
" Jane caught a bad cold , pdooxr thing !
" Jane caught a bad cold , poor thing !
" It is a great pleasure where one can indulge in it ," said the young man , " though there are not many houses that I should presume on so far ; but in coming _home_ I felt I might do any thing ."
" It is a 

In [ ]:
count

19

In [ ]:
p['worth',"which"]

In [ ]:
emma_words.count('Mr')

In [ ]:
def input_test(sents,words,p):

  unique_words = set(words)
  # test = sents[random.randint(0,len(sents)-1)]
  # print(*test)
  # rand_ind = random.randrange(0,len(test))
  # temp = (edits0(test[rand_ind]).union(edits1(test[rand_ind])).union(edits2(test[rand_ind])))
  # real_nonreal = random.randint(1,5)
  # if real_nonreal == 1:
  #   temp = temp.intersection(unique_words)
  # else:
  #   temp = temp.difference(unique_words)
  # temp = list(temp)
  # chosen_index = random.randrange(0,len(temp))
  # test.pop(rand_ind)
  # test.insert(rand_ind,temp[chosen_index])

  test = input()
  if test[-1]=='.':
    test = test[:-1]
  test = list(test.split())


  ind1 = 0
  temp1 = 1
  for i in range(len(test)-1):
    if p[test[i],test[i+1]] < temp1:
      ind1 = i
      temp1 = p[test[i],test[i+1]]
  
  ind2 = 0
  temp2 = 1
  for i in range(1,len(test)):
    if p[test[len(test)-i-1],test[len(test)-i]] < temp2:
      ind2 = len(test)-i
      temp2 = p[test[len(test)-i-1],test[len(test)-i]]

  if ind2-ind1 ==2:
    ind = [ind2-1]
  else:
    ind = []
    for i in range(min(ind1,ind2),max(ind1,ind2)+1):
      ind.append(i)
  
  print("ind:" ,ind)

  alphabet = 'abcdefghijklmnopqrstuvwxyz'

  candidates = dict()

  for i in ind:
    candidates[i] = (edits0(test[i]).union(edits1(test[i])).union(edits2(test[i])))

  
  for i in range(len(candidates)):
    temp = candidates[ind[i]].intersection(unique_words)
    temp = list(temp)
    candidates[ind[i]] = temp

  ind_max = 0
  temp = 0
  candidate = ''


  for index in ind:
    if index != 0 and index != len(test)-1:
      for i in range(len(candidates[index])):
        if p[test[index-1],candidates[index][i]]*p[candidates[index][i],test[index+1]] > temp:
          candidate = candidates[index][i]
          temp = p[test[index-1],candidates[index][i]]*p[candidates[index][i],test[index+1]]
          ind_max = index
    elif index == 0:
      for i in range(len(candidates[index])):
        if p[candidates[index][i],test[index+1]] > temp:
          candidate = candidates[index][i]
          temp = p[candidates[index][i],test[index+1]]
          ind_max = index
    else:
      for i in range(len(candidates[index])):
        if p[test[index-1],candidates[index][i]] > temp:
          candidate = candidates[index][i]
          temp = p[test[index-1],candidates[index][i]]
          ind_max = index
  # if real_nonreal == 1:
  #   print("real word error")
  # else: 
  #   print("nonreal word error")
  print(ind_max)
  print(candidate)
  print("origin: ", *test)
  test.pop(ind_max)
  test.insert(ind_max,candidate)
  print("corrected: ", *test)







In [ ]:
input_test(emma_sents,emma_words,p)

In [ ]:
test = input()

In [ ]:
if test[-1]=='.':
  test = test[:-1]
test = list(test.split())

In [ ]:
print(test)

In [ ]:
ind1 = 0
temp1 = 1
for i in range(len(test)-1):
  if p[test[i],test[i+1]] < temp1:
    ind1 = i
    temp1 = p[test[i],test[i+1]]

In [ ]:
ind1

In [ ]:
ind2 = 0
temp2 = 1
for i in range(1,len(test)):
  if p[test[len(test)-i-1],test[len(test)-i]] < temp2:
    ind2 = len(test)-i
    temp2 = p[test[len(test)-i-1],test[len(test)-i]]

In [ ]:
ind2

In [ ]:
if ind2-ind1 ==2:
  ind = [ind2-1]
else:
  ind = []
  for i in range(min(ind1,ind2),max(ind1,ind2)+1):
    ind.append(i)


In [ ]:
ind

In [ ]:
candidates = dict()

In [ ]:
for i in ind:
  candidates[i] = (edits0(test[i]).union(edits1(test[i])).union(edits2(test[i])))

In [ ]:
unique_words = set(emma_words)
for i in range(len(candidates)):
  temp = candidates[ind[i]].intersection(unique_words)
  temp = list(temp)
  candidates[ind[i]] = temp
candidates

In [ ]:
ind_max = 0
temp = 0
candidate = ''


for index in ind:
  if index != 0 and index != len(test)-1:
    for i in range(len(candidates[index])):
      if p[test[index-1],candidates[index][i]]*p[candidates[index][i],test[index+1]] > temp:
        candidate = candidates[index][i]
        temp = p[test[index-1],candidates[index][i]]*p[candidates[index][i],test[index+1]]
        ind_max = index
  elif index == 0:
    for i in range(len(candidates[index])):
      if p[candidates[index][i],test[index+1]] > temp:
        candidate = candidates[index][i]
        temp = p[candidates[index][i],test[index+1]]
        ind_max = index
  else:
    for i in range(len(candidates[index])):
      if p[test[index-1],candidates[index][i]] > temp:
        candidate = candidates[index][i]
        temp = p[test[index-1],candidates[index][i]]
        ind_max = index

print(candidate)
print(ind_max)

In [ ]:
test.pop(ind_max)
test.insert(ind_max,candidate)
print(*test)